In [1]:
import tsdm

In [2]:
tsdm.load_model("ODE-RNN")

In [3]:
from pathlib import Path
from types import ModuleType
import sys
import importlib
from contextlib import contextmanager


@contextmanager
def add_to_path(p: Path) -> None:
    """Source: https://stackoverflow.com/a/41904558/9318372"""
    old_path = sys.path
    sys.path = sys.path[:]
    sys.path.insert(0, str(p))
    try:
        yield
    finally:
        sys.path = old_path


def path_import(module_path: Path, module_name: str = None) -> ModuleType:
    """
    implementation taken from https://docs.python.org/3/library/importlib.html#importing-a-source-file-directly
    Source: https://stackoverflow.com/a/41904558/9318372

    Parameters
    ----------
    module_path: Path
        Path to the folder where the module is located
    module_name: str, optional

    Returns
    -------
    """

    module_name = module_name or module_path.parts[-1]
    module_init = module_path.joinpath("__init__.py")
    assert module_init.exists(), F"Module {module_path} has no __init__ file !!!"

    with add_to_path(module_path):
        spec = importlib.util.spec_from_file_location(module_name, str(module_init))
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
        return module

In [29]:
module = path_import(Path("/home/rscholz/.tsdm/models/ODE-RNN"))

create_net = module.lib.utils.create_net
ODEFunc = module.lib.ode_func.ODEFunc
DiffeqSolver = module.lib.diffeq_solver.DiffeqSolver
ODE_RNN = module.lib.ode_rnn.ODE_RNN


In [ ]:
ode_func_net = utils.create_net(
    n_ode_gru_dims, n_ode_gru_dims, n_layers = args.rec_layers, n_units = args.units, nonlinear = nn.Tanh)

rec_ode_func = ODEFunc(
    input_dim = input_dim, 
    latent_dim = n_ode_gru_dims,
    ode_func_net = ode_func_net,
    device = device).to(device)

z0_diffeq_solver = DiffeqSolver(input_dim, rec_ode_func, "euler", args.latents, 
    odeint_rtol = 1e-3, odeint_atol = 1e-4, device = device)

model = ODE_RNN(input_dim, n_ode_gru_dims, device = device, 
    z0_diffeq_solver = z0_diffeq_solver, n_gru_units = args.gru_units,
    concat_mask = True, obsrv_std = obsrv_std,
    use_binary_classif = args.classif,
    classif_per_tp = classif_per_tp,
    n_labels = n_labels,
    train_classif_w_reconstr = (args.dataset == "physionet")
    ).to(device)

In [4]:
x = tsdm.load_dataset('electricity')
